In [2]:
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os.path

data_sufix = 'data/'

In [3]:
# configure Session class with desired options
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')

In [90]:
engine.table_names()

['hashes', 'hashtag', 'retweet', 'tweet', 'tweetToHashes', 'user', 'vader']

In [5]:
hashtags = ['h1b', 'usvisa', 'immigrationreform', 'illegalImmigration', 'immigration',
'immigrantcaravan', 'caravaninvasion', 'buildthewall']
remaining_hashtags = ['illegalaliens', 'birthrightcitizenship']
hashtagsDict = {'name': hashtags}

In [5]:
hashtagData = pd.DataFrame.from_dict(hashtagsDict)
hashtagData.to_sql('hashtag', engine, if_exists='replace', index=True, index_label='id')

In [11]:
for hashtag in hashtags:
    
    hashtag_id = pd.read_sql_query('SELECT id FROM hashtag WHERE hashtag.name="'+hashtag+'"', engine).values[0][0]

    tweetData = pd.read_csv(data_sufix+hashtag+'-tweet.csv', 
                            usecols=['tweet_id','tweet_text','tweet_created_at','tweet_retweet_count',
                                     'tweet_favorite_count','user_id'])
    tweetData['hashtag_id'] = hashtag_id
    
    retweetData = pd.read_csv(data_sufix+hashtag+'-retweet.csv', 
                              usecols=['retweet_id','source_tweet_id','retweet_text','retweet_created_at','user_id'])
    
    tweetData.to_sql('tweet', engine, if_exists='append')
    retweetData.to_sql('retweet', engine, if_exists='append')

In [12]:
userData = pd.read_csv(data_sufix+'user.csv', 
                        usecols=['user_id','user_name','user_location','user_friends_count','followers_count'])

userData.to_sql('user', engine, if_exists='append')

In [42]:
#HASHTAGS

allTweets = pd.read_sql_query('SELECT tweet_id as id, tweet_text as text FROM tweet', engine)
print(allTweets.shape)
allRetweets =  pd.read_sql_query('SELECT retweet_id as id, retweet_text as text FROM retweet', engine)
print(allRetweets.shape)
textData = allTweets.append([allRetweets])
textData.drop_duplicates(inplace=True)
# textData = textData[:50000]
textData

(17220, 2)
(77828, 2)


,id,text
0,1056095192930025472,b'@Earlybird0073 @immivoice I guess #h1b #h4ea...
1,1056125697482256384,b'@AmerMedicalAssn @USCIS #kansas #traitor @Ke...
2,1056125759851556864,b'@AmerMedicalAssn @USCIS @KevinYoder sneaked ...
3,1056125800196595712,b'@AmerMedicalAssn @USCIS #HR392 has been snea...
4,1056125844966592512,b'@AmerMedicalAssn @USCIS 150 year #Backlog of...
5,1056126058175647744,b'@AmerMedicalAssn @USCIS This is how #Indians...
6,1056138781626118144,b'@USCIS #Indians have #monopolized #H1B #visa...
7,1056152900727119873,"b'@ghosh15728, article claims ""acute talent cr..."
8,1056153754863652864,b'U.S. Citizenship and #Immigration Services i...
9,1056165457491906561,b'@EllieMasoom @America_Desi @pratikchhatbar @...


In [43]:
import re
hashes=''
regex = r"#([a-zA-Z\d]*)"
for index, row in textData.iterrows():
    match = re.findall(regex, row["text"])
    hashes = list(set(hashes)) + [x.lower() for x in match]
# print(hashes)
len(hashes)

['', 'democratshaveanantiamericanagenda', 'asterixandobelix', 'pres', 'r', 'attemtedmurder', 'indivisible', 'trumpatwar', 'pro2nd', 'boycottnetfl', 'buildthedamnwall', 'buildtha', 'facebook', 'french', 'draintheswam', 'openborder', 'caravanadelm', 'default', 'closetheborde', 'obamacrimes', 'pasadena', 'istandwithsarah', 'islamexposed', 'censored', 'weareallillegal', 'invasion', 'guardourborders', 'drainthe', 'immigrantcaravans', 'easydecisions', 'charlottedornellas', 'afghanistanelection2018', 'statueoflibertyday', 'ny27', 'dna', 'ethics', 'america4israel', 'migrations', 'nomeansno', 'noanchorbabies', 'ustroops', 'marrakechdeclaration', 'novem', 'liberalsaretermites', 'immigrate', 'caravangoback', 'invasionfromthesouth', 'youtube', 'uspoli', 'inclusion', 'watch', 'stopthebias', 'bacon', 'stopillegaldrugtrafficking', 'calif', 'consult', 'stopthecarava', 'morocco', 'senate', 'sgomberi', 'dontgetmadvotered', 'militarydeployed', 'algeria', 'stoptillegals', 'georgiagop', 'tucson', 'impeach4

In [47]:
allHashes = pd.DataFrame({'hashtag':hashes})
allHashes.to_sql('hashes', engine, if_exists='append')

In [76]:
allDBHashes = pd.read_sql_query('SELECT * FROM hashes', engine)
print(allDBHashes.shape)
allDBHashes[:5]

(5244, 2)


,index,hashtag
0,0,
1,1,democratshaveanantiamericanagenda
2,2,asterixandobelix
3,3,pres
4,4,r


In [87]:
import re
hashes=''
regex = r"#([a-zA-Z\d]*)"
tweetToHashes = pd.DataFrame(columns=['tweet_id', 'hash_id'])
for index, row in textData.iterrows():
    match = re.findall(regex, row["text"])
    hashes = [x.lower() for x in match]
    tweetID = row["id"]
    for hsh in hashes:
        hashID = allDBHashes[allDBHashes['hashtag'] == hsh].iloc[0]["index"]
        tweetToHashes.loc[len(tweetToHashes)]=[tweetID, int(hashID)]
tweetToHashes

,tweet_id,hash_id
0,1056095192930025472,1751
1,1056095192930025472,1632
2,1056095192930025472,4653
3,1056125697482256384,1263
4,1056125697482256384,934
5,1056125697482256384,2020
6,1056125697482256384,3524
7,1056125697482256384,5111
8,1056125697482256384,1751
9,1056125697482256384,238


In [88]:
tweetToHashes.to_sql('tweetToHashes', engine, if_exists='append')

In [89]:
# associate it with our custom Session class
Session.configure(bind=engine)